In [1]:
# Import packages
import os
import tweepy as tw
import pandas as pd
import numpy as np
import re 
#from geopy.geocoders import Nominatim

# Set keys
consumer_key= 'svKJvnLZnXayRm2frd4DduozL'
consumer_secret= 'mfdYmky8LVpBUyj5yv1xTGHwc4O4QTIfhh6uo0VBjnHA4AsL5e'
access_token= '1331337099786776576-EDuVu6acOiOTGFHn0LsC6wgNPOlNzI'
access_token_secret= '8oQ6HJEOiWexEueOlUdT5YBslzvc8qdN3aZP8ZiOssiWS'

# Set date search
date_since = "2020-12-20"
date_to = '2020-12-26'

run = '2020-12-20-26'
# Set date search #yyyyMMddHHmm
#date_since = '202001010000'
#date_to = '202012252359'


folder_name = 'data/'
# Create data folder
if not os.path.isdir('./'+folder_name):
    os.mkdir('./'+folder_name)
    
tweet_file_name = folder_name+run+"_tweet.xlsx"
retweet_file_name = folder_name+run+"_retweet.xlsx"

In [2]:
# Create cities list of Italy
df_city = pd.read_excel('useful_datasets/italian_city_dataset.xlsx',engine='openpyxl')
df_city['city'] = df_city['city'].str.lower()
df_city['region'] = df_city['region'].str.lower()
df_city['province'] = df_city['province'].str.lower()

cities                  = list(df_city['city'])
regions_of_cities       = list(df_city['region'])
codes_regions_of_cities = list(df_city['region_code'])
provinces               = list(df_city['province'])
lat_provinces           = list(df_city['lat'])
lon_provinces           = list(df_city['lon'])
car                     = list(df_city['car'])

# Create region list of Italy
df_region = pd.read_excel('useful_datasets/italian_regions_dataset.xlsx',engine='openpyxl',index_col=0,header=[0])
code_regions = list(df_region['region_code'])
regions      = list(df_region['region'])
lat_regions  = list(df_region['lat'])
lon_regions  = list(df_region['lon'])

In [3]:
# Set words to search
words = ['#novaccinoobbligatorio','#iononmivaccino','#nonvaccinatevi',
        '#noobbligovaccinale','#noobbligovaccino']

#words = ['#caosvax','#dannidavaccino','#autismoevaccini']
         
search_words = ''

for index in range(len(words)):
    if index==(len(words)-1):
        search_words = search_words + words[index] 
    else:
        search_words = search_words + words[index] + ' OR '
        
print(search_words)

#novaccinoobbligatorio OR #iononmivaccino OR #nonvaccinatevi OR #noobbligovaccinale OR #noobbligovaccino


In [4]:
# Access with API
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

N = 1000

# Search tweets
tweets = tw.Cursor(api.search,
                       q=search_words+"-filter:retweets", # Filter for no retweets!
                       lang="it",
                       since=date_since,
                       until = date_to).items(N)

#tweets = tw.Cursor(api.search_full_archive,
#                   environment_name='novaccino',
#                   query=search_words, 
#                   fromDate=date_since,
#                   toDate = date_to).items(N)

#tweets = api.search_full_archive(environment_name='novaccino',
#                                 query=search_words, 
#                                 fromDate=date_since,
#                                 toDate = date_to,
#                                 maxResults=N)


# Store relevant informations
tweets_info = [[ tweet.user.screen_name, tweet.user.id, tweet.user.description, tweet.user.location, 
                 tweet.user.followers_count, tweet.user.favourites_count, tweet.user.created_at, 
                tweet.id, tweet.created_at, tweet.text, tweet.favorite_count, 
                tweet.retweet_count, tweet.in_reply_to_screen_name] for tweet in tweets]

# Convert relevant informations to a Pandas dataframe
columns = [["user","user","user","user","user","user","user",
         "tweet","tweet","tweet","tweet","tweet","tweet"],
         ["screen_name", "id", "description", "location", "followers_count", "favourites_count", "created_at",
          "id", "created_at", "text","favorites_count","retweet_count","in_reply_to_screen_name"]]

tweet_data = pd.DataFrame(data=tweets_info,
                          columns=columns)

dim = tweet_data.shape

print('Dim before:',dim)

Dim before: (97, 13)


In [5]:
#tweet_data.isnull().values.any()
tweet_data = tweet_data.replace(np.nan, 'None', regex=True)

In [6]:
# Lower the characheters and remove some words
tweet_data[("user","location")] = tweet_data[("user","location")].str.lower()
words = ['italia','italy','','terra','earth']
tweet_data = tweet_data[~tweet_data[("user","location")].isin(words)]

# Replace all the special characters with a null string
tweet_data[("user","location")] = tweet_data[("user","location")].str.replace(r'[\-\!\@\#\$\%\^\&\*\(\)\_\+\[\]\;\'\.\,\/\{\}\:\"\<\>\?]' , '')

dim = tweet_data.shape

print('Dim after:', dim)

Dim after: (47, 13)


In [197]:
#result = [print(x) for x in tweet_data[('tweet','text')]]

In [7]:
# If present, extract region from location information
for i in range(len(regions)):
    mask = tweet_data[("user","location")].str.contains(fr'\b{regions[i]}\b', regex=True, case=False)
    tweet_data.loc[mask,("user","code_region")] = code_regions[i] 
    tweet_data.loc[mask,("user","region")]      = regions[i]   
    tweet_data.loc[mask, ("user","lat_region")] = lat_regions[i]
    tweet_data.loc[mask, ("user","lon_region")] = lon_regions[i]
    #tweet_data.loc[mask, 'location'] = tweet_data.loc[mask, 'location'].str.replace(reg,'')

# If present, extract province from location information
for i in range(len(cities)):
    mask = tweet_data[("user","location")].str.contains(fr'\b{cities[i]}\b', regex=True, case=False)
    tweet_data.loc[mask, ("user","city")]         = cities[i]
    tweet_data.loc[mask, ("user","province")]     = provinces[i]
    tweet_data.loc[mask, ("user","lat_province")] = lat_provinces[i]
    tweet_data.loc[mask, ("user","lon_province")] = lon_provinces[i]
    tweet_data.loc[mask, ("user","region")]       = regions_of_cities[i] 
    tweet_data.loc[mask, ("user","code_region")]  = codes_regions_of_cities[i]

# If present, extract province information from car
for i in range(len(car)):
    mask = tweet_data[("user","location")].str.contains(fr'\b{car[i]}\b', regex=True, case=False)
    tweet_data.loc[mask, ("user","province")]     = provinces[i]
    tweet_data.loc[mask, ("user","lat_province")] = lat_provinces[i]
    tweet_data.loc[mask, ("user","lon_province")] = lon_provinces[i]
    tweet_data.loc[mask, ("user","region")]       = regions_of_cities[i] 
    tweet_data.loc[mask, ("user","code_region")]  = codes_regions_of_cities[i]
    
# Remove row if 'region' & 'city' are null
tweet_data = tweet_data.drop(tweet_data[(tweet_data[("user","region")].isnull()) & (tweet_data[("user","city")].isnull())].index)

# Print dataset columns
#print(tweet_data.columns)

# Remove city None
tweet_data = tweet_data[~tweet_data[("user","location")].isin(['none'])]
print('Number of rows:', tweet_data.shape[0],'out of', dim[0], '(initial:',N,')')

# Convert tweet dataset into a excel file
tweet_data.to_excel(tweet_file_name)  

tweet_data.head(5)

Number of rows: 28 out of 47 (initial: 1000 )


user                       \
        screen_name                   id   
0       Lucio_freni             76904458   
3    pamelabelfanti            884245951   
9   Antonio76278652  1019635732267307011   
11  Antonio76278652  1019635732267307011   
12  Antonio76278652  1019635732267307011   

                                                                       \
                                          description        location   
0   #scrittore #Asperger   #photographer #trader R...            roma   
3   La trasgressione non è peccato....... libera d...       follonica   
9                                                      genova liguria   
11                                                     genova liguria   
12                                                     genova liguria   

                                                                       tweet  \
   followers_count favourites_count          created_at                   id   
0             4184            18077 2009-09-24 10:05:51  1342620974534029315   
3               80              112 2012-10-16 11:23:25  1342595159490490372   
9              292            29223 2018-07-18 17:31:17  1342581703068569608   
11             292            29223 2018-07-18 17:31:17  1342560022971240450   
12             292            29223 2018-07-18 17:31:17  1342549753289125890   

                                                                           \
            created_at                                               text   
0  2020-12-25 23:59:19  Però  per arrivare esattamente il #6gennaio, a...   
3  2020-12-25 22:16:44  Ogni giorno che passa mi pongo sempre la stess...   
9  2020-12-25 21:23:16  @TwittGiorgio @YouTube Infatti Giorgio #iononm...   
11 2020-12-25 19:57:07  @POPOLOdiTWlTTER E se ci saranno chi paga sto ...   
12 2020-12-25 19:16:19  @51inif In culo a loro #iononmivaccino nel dub...   

    ...                                              user                      \
    ... retweet_count in_reply_to_screen_name code_region   region lat_region   
0   ...             0                    None        12.0    lazio        NaN   
3   ...             0                    None         9.0  toscana        NaN   
9   ...             0            TwittGiorgio         7.0  liguria   44.40726   
11  ...             0         POPOLOdiTWlTTER         7.0  liguria   44.40726   
12  ...             0                  51inif         7.0  liguria   44.40726   

                                                              
   lon_region       city  province lat_province lon_province  
0         NaN       roma      roma     41.89332    12.482932  
3         NaN  follonica  grosseto     42.77511    11.287804  
9    8.933862     genova    genova     44.40726     8.933862  
11   8.933862     genova    genova     44.40726     8.933862  
12   8.933862     genova    genova     44.40726     8.933862  

[5 rows x 21 columns]

In [10]:
# Column of dataframe for retweets
columns = [["user","user","user","user","user","user","user",
         "tweet","tweet","tweet","tweet","tweet","tweet"],
         ["screen_name", "id", "description", "location", "followers_count", "favourites_count", "created_at",
          "id_original","id", "created_at", "text","favorites_count","retweet_count"]]

# Get tweets id
tweet_id = np.array(tweet_data[("tweet","id")])

retweet_data_list = []

for ID in tweet_id:
    
    mask = tweet_data[('tweet','id')]==ID
    
    if (tweet_data.loc[mask, ('tweet','retweet_count')].values == 0) :
        continue
    else: 
        # getting the retweeters 
        retweets_list = api.retweets(ID) 
   
        # Store relevant informations of retweets
        retweets_info = [[tweet.user.screen_name, tweet.user.id, tweet.user.description, tweet.user.location, 
                          tweet.user.followers_count, tweet.user.favourites_count, tweet.user.created_at, 
                         ID, tweet.id, tweet.created_at, tweet.text, tweet.favorite_count, tweet.retweet_count] for tweet in retweets_list]
    
        data = pd.DataFrame(data=retweets_info,
                          columns=columns)

        retweet_data_list.append(data)

retweet_data = pd.concat(retweet_data_list,ignore_index=True)

In [11]:
# If present, extract region from location information
for i in range(len(regions)):
    mask = retweet_data[("user","location")].str.contains(fr'\b{regions[i]}\b', regex=True, case=False)
    retweet_data.loc[mask,("user","code_region")] = code_regions[i] 
    retweet_data.loc[mask,("user","region")] = regions[i]   
    retweet_data.loc[mask, ("user","lat_region")] = lat_regions[i]
    retweet_data.loc[mask, ("user","lon_region")] = lon_regions[i]
    #tweet_data.loc[mask, 'location'] = tweet_data.loc[mask, 'location'].str.replace(reg,'')

# If present, extract province from location information
for i in range(len(cities)):
    mask = retweet_data[("user","location")].str.contains(fr'\b{cities[i]}\b', regex=True, case=False)
    retweet_data.loc[mask, ("user","city")]         = cities[i]
    retweet_data.loc[mask, ("user","province")]     = provinces[i]
    retweet_data.loc[mask, ("user","lat_province")] = lat_provinces[i]
    retweet_data.loc[mask, ("user","lon_province")] = lon_provinces[i]
    retweet_data.loc[mask, ("user","region")]       = regions_of_cities[i] 
    retweet_data.loc[mask, ("user","code_region")]  = codes_regions_of_cities[i]

# If present, extract province information from car
for i in range(len(car)):
    mask = retweet_data[("user","location")].str.contains(fr'\b{car[i]}\b', regex=True, case=False)
    retweet_data.loc[mask, ("user","province")]     = provinces[i]
    retweet_data.loc[mask, ("user","lat_province")] = lat_provinces[i]
    retweet_data.loc[mask, ("user","lon_province")] = lon_provinces[i]
    retweet_data.loc[mask, ("user","region")]       = regions_of_cities[i] 
    retweet_data.loc[mask, ("user","code_region")]  = codes_regions_of_cities[i]
        
# Remove row if 'region' & 'city' are null
retweet_data = retweet_data.drop(retweet_data[(retweet_data[("user","region")].isnull()) & (retweet_data[("user","city")].isnull())].index)

# Remove city None
retweet_data = retweet_data[~retweet_data[("user","location")].isin(['none'])]
print('Number of rows:', retweet_data.shape[0],'out of', dim[0], '(initial:',N,')')

# Convert tweet dataset into a excel file
retweet_data.to_excel(retweet_file_name)  

retweet_data

Number of rows: 10 out of 47 (initial: 1000 )


user                       \
        screen_name                   id   
1    Marylu58121349   903357817020706816   
2        angy_cocco             53074807   
3         _Khassia_             45782707   
6      Lara86124487  1262000545214185473   
8     RitaB66743159  1082807497415639041   
14          Gingio5   869963379204608000   
15     LuisaPistini  1196018762593525765   
18  VittorioFerram1  1223200116670115850   
19       lauramjj58            137120635   
21         DavPoggi  1241675979929661442   

                                                       \
                                          description   
1   Covid: psico pseudo Pandemia\n#SpegnilaTv\n#Ac...   
2   P&P #PenadellePersiane per servirvi\n\nMembra ...   
3   - La plebe vuol essere stordita da invettive, ...   
6   Sono contro il GOVERNO GIALLOROSSO👎NO UE! \nbl...   
8   Non amo definirmi con parole....Troppo restrit...   
14  Ostico e agnostico, anacoreta harleyciclista c...   
15  Solo due cose sono infinite l’universo e la st...   
18  Rispetto la Costituzione Repubblicana,orgoglio...   
19  I LOVE LIFE..MY FAMILY FRIENDS...LOVE MUSIC AN...   
21   Avvocato civilista viaggiatore libero e liberale   

                                                               \
                    location followers_count favourites_count   
1          Nápoles, Campania             639            53989   
2                     Milano            1689            57848   
3            Liguria, Italia            4815            41943   
6             Verona, Veneto             806             8068   
8              Italia Marche            2730           115248   
14         Lombardia, Italia            1213            30047   
15         Lombardia, Italia            1185            43699   
18           Roma, Italia 🇮🇹             488            15595   
19  Milano  and  Los Angeles             995            86052   
21          Napoli, Campania             346            18724   

                                      tweet                       \
            created_at          id_original                   id   
1  2017-08-31 20:44:21  1342533456002228225  1342549453551636480   
2  2009-07-02 14:20:02  1342533456002228225  1342537742736105473   
3  2009-06-09 06:36:49  1342055714727096320  1342057232880558080   
6  2020-05-17 12:42:55  1340774647776899073  1340778819586801665   
8  2019-01-09 01:13:19  1340715773581545475  1340775721749405696   
14 2017-05-31 17:06:47  1340681023160520705  1340794219997360128   
15 2019-11-17 10:54:40  1340681023160520705  1340779025871073280   
18 2020-01-31 11:03:44  1340681023160520705  1340731584694149120   
19 2010-04-25 21:34:30  1340681023160520705  1340731314371260416   
21 2020-03-22 10:40:04  1340551205999808512  1340571815048073217   

                        ...                                      user  \
            created_at  ... favorites_count retweet_count code_region   
1  2020-12-25 19:15:07  ...               0             3        15.0   
2  2020-12-25 18:28:35  ...               0             3         3.0   
3  2020-12-24 10:39:13  ...               0             1         7.0   
6  2020-12-20 21:59:15  ...               0             3         5.0   
8  2020-12-20 21:46:57  ...               0             3        11.0   
14 2020-12-20 23:00:27  ...               0            13         3.0   
15 2020-12-20 22:00:04  ...               0            13         3.0   
18 2020-12-20 18:51:34  ...               0            13        12.0   
19 2020-12-20 18:50:29  ...               0            13         3.0   
21 2020-12-20 08:16:41  ...               0             1        15.0   

                                                                                
       region lat_region lon_region    city province lat_province lon_province  
1    campania  40.835934  14.248783     NaN      NaN          NaN          NaN  
2   lombardia        NaN        NaN  milano   milano    45.466800     9.190500  
3 